In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import PIL


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
train_images = np.expand_dims(x_train, -1).astype("float32") / 255
train_images = train_images.reshape(-1, 28, 28, 1)




#### Instantiate VAE and Checkpoint path

In [ ]:
checkpoint_path = '/Users/charlherbst/Charl/training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
vae = VAE(dim_z=latent_dim)


#### Training

In [ ]:
latent_dim = 8
lr = 0.0005
batch_size = 64
epochs = 40
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss=vae_loss)
train_history = vae.fit(x=train_images,y=train_images,batch_size=batch_size,
                        epochs=epochs, verbose=1, shuffle=True, callbacks = cp_callback)

#### Visualise reconstruction loss

In [ ]:
plt.plot(train_history.history['loss'])

#### Instantiate new VAE and load weights

In [ ]:
os.listdir(checkpoint_dir)
latent_dim = 8
vae = VAE(dim_z=latent_dim)
vae.load_weights(checkpoint_path)
_,image_encodings,_ = vae.encoder(train_images)


#### Visualise latent encodings

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
ax.scatter3D(image_encodings[:, 0], image_encodings[:, 1], image_encodings[:, 2], color = "blue")
plt.xlabel("z[0]")
plt.ylabel("z[1]")
plt.title("Image encodings in 3D of 10 MNIST digits")
plt.show()

#### Visualise data reconstruction

In [ ]:
image_decodings = vae.decoder(image_encodings)
image_decodings = np.reshape(np.array(image_decodings),input_data.shape)

plt.figure
plt.axis('off')
plt.imshow(image_decodings[0])
plt.show()